In [1]:
from dotenv import load_dotenv
import os
import itertools
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
from broker.futures_bybit_broker import FuturesBybitBroker


load_dotenv()

API_KEY = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

broker = FuturesBybitBroker(API_KEY, API_SECRET)

In [4]:
from risk_management.stop_loss.atr_stop_loss_finder import ATRStopLossFinder
from risk_management.stop_loss.low_high_stop_loss_finder import LowHighStopLossFinder
from risk_management.stop_loss.simple_stop_loss_finder import SimpleStopLossFinder
from risk_management.stop_loss.trailing_stop_loss_finder import TrailingStopLossFinder
from risk_management.take_profit.noop_take_profit_finder import NoopTakeProfitFinder
from risk_management.take_profit.risk_reward_take_profit_finder import RiskRewardTakeProfitFinder
from strategy.bollinger_engulfing_strategy import BollingerEngulfing
from strategy.engulfing_zlema_strategy import EngulfingSMA
from strategy.extreme_euphoria_bb_strategy import ExtremeEuphoriaBBStrategy
from strategy.kangaroo_tail_strategy import KangarooTailStrategy


symbols = [
    'ETHUSDT',
    'XRPUSDT',
    'BTCUSDT',
    'SOLUSDT'
]
timeframes = [
    '1m',
    '3m',
    # '5m'
]
row_number = 5000
strategies = [
    # SwingIndicatorStrategy(),
    BollingerEngulfing(),
    # AwesomeOscillatorBBStrategy(),
    EngulfingSMA(),
    KangarooTailStrategy(),
    ExtremeEuphoriaBBStrategy(),
    # StructuredDurationStrategy()
]

atr_stop_loss_finders = [ATRStopLossFinder(multiplier=multi) for multi in [0.88, 0.9, 1, 1.2, 1.5]]
highlow_loss_finders = [LowHighStopLossFinder(lookback_period=lookback) for lookback in [5, 10, 15, 20, 50, 100]]
trailing_stop_loss  = [TrailingStopLossFinder(atr_multiplier=multi) for multi in [0.88, 0.9, 1.0, 1.2]]
stop_loss_finders = [SimpleStopLossFinder()] + atr_stop_loss_finders + highlow_loss_finders + trailing_stop_loss

take_profit_finders = [RiskRewardTakeProfitFinder(risk_reward_ratio=reword) for reword in [1, 1.5, 2, 3, 4]] + [NoopTakeProfitFinder()]
combinations = list(itertools.product(symbols, timeframes, strategies, stop_loss_finders, take_profit_finders))
results_list = []

In [5]:
from backtest.backtester import Backtester
from risk_management.risk_manager import RiskManager
from shared.trade_type import TradeType


for symbol, timeframe, strategy, stop_loss_finder, take_profit_finder in tqdm(combinations):
    ohlcv = broker.get_historical_data(symbol, timeframe, limit=row_number)
    stop_loss_finder.set_ohlcv(ohlcv)
    rm = RiskManager(stop_loss_finder, take_profit_finder)
    backtester = Backtester(strategy, rm, ohlcv)
    result = backtester.run(trade_type=TradeType.BOTH)
    result.update({
        'symbol': symbol,
        'timeframe': timeframe,
        'strategy': strategy,
        'stop_loss_finder': stop_loss_finder,
        'take_profit_finder': take_profit_finder,
        'win_rate': result['win_rate'] * 100
    })
    results_list.append(result)
    print(f"{symbol} orders for strategy: {strategy}, TimeFrame: {timeframe}, StopLossFinder: {stop_loss_finder}, TakeProfitFinder: {take_profit_finder}")
    print(backtester.get_orders().head())

  0%|          | 0/3072 [00:00<?, ?it/s]

ETHUSDT orders for strategy: BollingerEngulfing, TimeFrame: 1m, StopLossFinder: SimpleStopLossFinder(stop_loss_pct=0.02), TakeProfitFinder: RiskRewardTakeProfitFinder(risk_reward_ratio=1)
    side    entry     exit    profit  stop loss  take profit
0   long  1744.96  1789.89  12.76012   1710.061     1779.859
1   long  1772.00  1722.49 -14.01133   1736.560     1807.440
2  short  1710.20  1744.07  -9.78843   1744.404     1675.996
ETHUSDT orders for strategy: BollingerEngulfing, TimeFrame: 1m, StopLossFinder: SimpleStopLossFinder(stop_loss_pct=0.02), TakeProfitFinder: RiskRewardTakeProfitFinder(risk_reward_ratio=1.5)
   side    entry     exit   profit  stop loss  take profit
0  long  1744.96  1710.20 -9.87184   1710.061     1797.309
1  long  1707.44  1734.37  7.72891   1673.291     1758.663
ETHUSDT orders for strategy: BollingerEngulfing, TimeFrame: 1m, StopLossFinder: SimpleStopLossFinder(stop_loss_pct=0.02), TakeProfitFinder: RiskRewardTakeProfitFinder(risk_reward_ratio=2)
   side    en

In [24]:
results_df = pd.DataFrame(results_list)
sorded_results = results_df.sort_values(by="total_pnl", ascending=False)

In [25]:
sorded_results.head()

,total_trades,successful_trades,win_rate,rate_of_return,total_pnl,average_pnl,sharpe_ratio,max_consecutive_wins,max_consecutive_losses,max_drawdown,symbol,timeframe,strategy,stop_loss_finder,take_profit_finder,use_trailing_stop_loss
16,7,4,57.142857,0.156743,156.743027,22.391861,0.493549,2,1,0.016789,XRPUSDT,1m,EngulfingSMA(fast_sma_period=21),"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
47,1,1,100.000000,0.105229,105.228820,105.228820,NaN,1,0,0.000000,SOLUSDT,1m,ExtremeEuphoriaBBStrategy,"LowHighStopLossFinder(stop_loss_pct=0.02, look...",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
46,1,1,100.000000,0.104078,104.077520,104.077520,NaN,1,0,0.000000,SOLUSDT,1m,ExtremeEuphoriaBBStrategy,"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
7,1,1,100.000000,0.084996,84.996100,84.996100,NaN,1,0,0.000000,ETHUSDT,1m,"KangarooTailStrategy(lookback=200, sma_period=...","ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
10,1,1,100.000000,0.082508,82.507600,82.507600,NaN,1,0,0.000000,ETHUSDT,1m,ExtremeEuphoriaBBStrategy,"ATRStopLossFinder(multiplier=0.9, atr_period=14)",RiskRewardTakeProfitFinder(risk_reward_ratio=1.5),False
